# HEATHOTSAUCE.COM SCRAPE

https://heathotsauce.com

In [1]:
import requests
from bs4 import BeautifulSoup

import pandas as pd

In [2]:
# scrape all 13 pages to get names and urls

names = []
urls = []

# go to each page (print progress)
for i in range(1, 14):
    url = 'https://heathotsauce.com/collections/all?page=' + str(i)
    page = requests.get(url)
    print i, page
    soup = BeautifulSoup(page.content, 'html.parser')
    
    #in each page, scrape the names
    for j in soup.findAll('span', class_='title product-name-custom'):
        names.append(j.get_text())
    for k in soup.findAll(itemprop = 'url'):
        urls.append(k.get('href'))

1 <Response [200]>
2 <Response [200]>
3 <Response [200]>
4 <Response [200]>
5 <Response [200]>
6 <Response [200]>
7 <Response [200]>
8 <Response [200]>
9 <Response [200]>
10 <Response [200]>
11 <Response [200]>
12 <Response [200]>
13 <Response [200]>


In [3]:
#combine names and urls into a df
names_urls = pd.DataFrame({'names':names, 'urls':urls})

# add url to start of urls
names_urls['urls'] = names_urls['urls'].apply(lambda x: 'https://heathotsauce.com' + str(x))

In [9]:
## Go to each unique url, grab name, price, manufacturer, description

## NOTE: I requested too many timesand was timed out. 
## Do this in batches, save as different df, then concat together

name = []
mfc = []
price = []
desc = []
urls = []

for n, url in enumerate(names_urls['urls'][400:]):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    
#     name
    try:
        name.append(soup.find('h1', class_='product_name').get_text())
    except:
        name.append('')
#     manufacturer
    try:
        mfc.append(soup.find('p', class_='vendor').get_text())
    except:
        mfc.append('')
#     price
    try:
        price.append(soup.find('span', class_='money').get_text())
    except:
        price.append('')
#     description
    try:
        desc.append(soup.find('div', class_='description').get_text())
    except:
        desc.append('')
#     url
    try:
        urls.append(url)
    except:
        urls.append('')
    
#     print progress, if response != 200, stop kernel. you have been timed out
    print n, page

df3 = pd.DataFrame({'names':name,
                   'mfc':mfc,
                   'price':price,
                   'desc':desc,
                   'url':urls})

0 <Response [200]>
1 <Response [200]>
2 <Response [200]>
3 <Response [200]>
4 <Response [200]>
5 <Response [200]>
6 <Response [200]>
7 <Response [200]>
8 <Response [200]>
9 <Response [200]>
10 <Response [200]>
11 <Response [200]>
12 <Response [200]>
13 <Response [200]>
14 <Response [200]>
15 <Response [200]>
16 <Response [200]>
17 <Response [200]>
18 <Response [200]>
19 <Response [200]>
20 <Response [200]>
21 <Response [200]>
22 <Response [200]>
23 <Response [200]>
24 <Response [200]>
25 <Response [200]>
26 <Response [200]>
27 <Response [200]>
28 <Response [200]>
29 <Response [200]>
30 <Response [200]>
31 <Response [200]>
32 <Response [200]>
33 <Response [200]>
34 <Response [200]>
35 <Response [200]>
36 <Response [200]>
37 <Response [200]>
38 <Response [200]>
39 <Response [200]>
40 <Response [200]>
41 <Response [200]>
42 <Response [200]>
43 <Response [200]>
44 <Response [200]>
45 <Response [200]>
46 <Response [200]>
47 <Response [200]>
48 <Response [200]>
49 <Response [200]>
50 <Respon

In [10]:
# combine all df batches. strip manufacturers of \n
full_df = pd.concat([df1, df2, df3])
full_df['mfc'] = full_df['mfc'].apply(lambda x: x.strip())

# remove gift sets and empty rows
full_df = full_df[full_df['mfc'] != 'Gift Set']
full_df = full_df[full_df['desc'] != '']
full_df.reset_index(inplace=True, drop=True)

# save df as csv
full_df.to_csv('hot_sauces.csv', encoding='utf-8', index=False)